In [1]:
import pandas as pd
from openai import OpenAI

In [ ]:
df = pd.read_csv("/content/yelp.csv")[["text", "stars"]].dropna()

test_df = df.sample(n=200, random_state=42)
train_df = df.drop(test_df.index)


In [ ]:
test_df.to_csv("test.csv")

In [ ]:
pip install google-generativeai


In [ ]:
pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 3.9 MB/s eta 0:00:00


 **PROMPT 1**


In [ ]:
import pandas as pd
import json
import time
from groq import Groq
import re
client = Groq(api_key="apikeyfromgroq")  # Get from https://console.groq.com/
df = pd.read_csv("/content/test (3).csv")


def get_prompt(review_text):
    prompt = f"""Classify this Yelp review into a rating from 1 to 5 stars.

Review: {review_text}

Respond ONLY with valid JSON:
{{"predicted_stars": <number>, "explanation": "<brief reason>"}}"""
    return prompt
def classify_review(review_text):
    try:
        prompt = get_prompt(review_text)

        response = client.chat.completions.create(
            model="llama-3.3-70b-versatile",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.3
        )

        content = response.choices[0].message.content
        json_match = re.search(r'\{[^{}]*"predicted_stars"[^{}]*\}', content)
        if json_match:
            result = json.loads(json_match.group())
            return result
        else:
            return {"error": "No JSON found", "predicted_stars": None, "explanation": ""}

    except Exception as e:
        return {"error": str(e), "predicted_stars": None, "explanation": ""}

results = []
for idx, row in df.iterrows():
    print(f"Processing {idx+1}/{len(df)}...", end='\r')

    response = classify_review(row['text'])

    results.append({
        "actual_stars": row["stars"],
        "predicted_stars": response.get("predicted_stars"),
        "explanation": response.get("explanation", ""),
        "correct": response.get("predicted_stars") == row["stars"]
    })

    time.sleep(0.1)

results_df = pd.DataFrame(results)

valid_predictions = results_df[results_df['predicted_stars'].notna()]
accuracy = valid_predictions['correct'].sum() / len(valid_predictions) if len(valid_predictions) > 0 else 0
json_validity = len(valid_predictions) / len(results_df)

print(f"\n\nResults:")
print(f"Total Reviews: {len(results_df)}")
print(f"Valid JSON Rate: {json_validity:.2%}")
print(f"Accuracy: {accuracy:.2%}")
print(f"Correct Predictions: {valid_predictions['correct'].sum()}")
results_df.to_csv("classification_results.csv", index=False)
print("\nSaved to: classification_results.csv")



Results:
Total Reviews: 200
Valid JSON Rate: 100.00%
Accuracy: 61.00%
Correct Predictions: 122

Saved to: classification_results.csv


**Prompt 2**

In [ ]:
import pandas as pd
import json
import time
from groq import Groq
import re

client = Groq(api_key="apikeyfromgroq")  # Get from https://console.groq.com/

df = pd.read_csv("/content/test (3).csv")
def get_prompt(review_text):
    prompt = f"""   your task will be take reviws form yelp a restaurants reviws understand it properly and by looking at the rewies you have tow tell how much stars(all the starts will be whole number) that person might have given .so basically you have to do opinion minnng.


Review: {review_text}

Respond ONLY with valid JSON:
{{"predicted_stars": <number>, "explanation": "<brief reason>"}}"""
    return prompt

def classify_review(review_text):
    try:
        prompt = get_prompt(review_text)

        response = client.chat.completions.create(
            model="llama-3.3-70b-versatile",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.3
        )

        content = response.choices[0].message.content
        json_match = re.search(r'\{[^{}]*"predicted_stars"[^{}]*\}', content)
        if json_match:
            result = json.loads(json_match.group())
            return result
        else:
            return {"error": "No JSON found", "predicted_stars": None, "explanation": ""}

    except Exception as e:
        return {"error": str(e), "predicted_stars": None, "explanation": ""}
results = []
for idx, row in df.iterrows():
    print(f"Processing {idx+1}/{len(df)}...", end='\r')

    response = classify_review(row['text'])

    results.append({
        "actual_stars": row["stars"],
        "predicted_stars": response.get("predicted_stars"),
        "explanation": response.get("explanation", ""),
        "correct": response.get("predicted_stars") == row["stars"]
    })

    time.sleep(0.1)

results_df = pd.DataFrame(results)
valid_predictions = results_df[results_df['predicted_stars'].notna()]
accuracy = valid_predictions['correct'].sum() / len(valid_predictions) if len(valid_predictions) > 0 else 0
json_validity = len(valid_predictions) / len(results_df)

print(f"\n\nResults:")
print(f"Total Reviews: {len(results_df)}")
print(f"Valid JSON Rate: {json_validity:.2%}")
print(f"Accuracy: {accuracy:.2%}")
print(f"Correct Predictions: {valid_predictions['correct'].sum()}")

results_df.to_csv("classification_results.csv", index=False)
print("\nSaved to: classification_results.csv")



Results:
Total Reviews: 200
Valid JSON Rate: 85.00%
Accuracy: 60.00%
Correct Predictions: 102

Saved to: classification_results.csv


**Prompt 3**

In [ ]:
import pandas as pd
import json
import time
from groq import Groq
import re
client = Groq(api_key="apikeyfromgroq")
df = pd.read_csv("/content/test (3).csv")
def get_prompt(review_text):
    prompt = f"""Your task is to analyze restaurant reviews taken from Yelp. Read each review carefully and understand the opinion being expressed. Based on the content, tone, and overall sentiment of the review, predict how many stars the reviewer likely gave. The rating should be a whole number only, from 1 to 5.

Use realistic review behavior:
• 5 stars = extremely enthusiastic, glowing praise, strong recommendations, or superlatives
• 4 stars = clearly positive but brief, casual, or focused on a few items without strong emphasis
• 3 stars = mixed or neutral
• 2 stars = mostly negative
• 1 star = very negative or angry

Review: {review_text}

You MUST respond with ONLY this exact JSON format (no other text before or after):
{{"predicted_stars": <number>, "explanation": "<brief reason>"}}"""
    return prompt

def classify_review(review_text):
    try:
        prompt = get_prompt(review_text)

        response = client.chat.completions.create(
            model="llama-3.3-70b-versatile",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.3,
            max_tokens=150
        )

        content = response.choices[0].message.content.strip()

        print(f"\nRaw response: {content[:200]}")

        try:
            result = json.loads(content)
            if "predicted_stars" in result:
                return result
        except:
            pass


        json_match = re.search(r'```(?:json)?\s*(\{[^`]*\})\s*```', content, re.DOTALL)
        if json_match:
            try:
                result = json.loads(json_match.group(1))
                if "predicted_stars" in result:
                    return result
            except:
                pass

        json_match = re.search(r'\{[^{}]*"predicted_stars"[^{}]*\}', content)
        if json_match:
            try:
                result = json.loads(json_match.group())
                if "predicted_stars" in result:
                    return result
            except:
                pass

        stars_match = re.search(r'"predicted_stars"\s*:\s*(\d)', content)
        if stars_match:
            return {
                "predicted_stars": int(stars_match.group(1)),
                "explanation": "Extracted from non-standard format"
            }

        return {"error": "No valid JSON found", "predicted_stars": None, "explanation": "", "raw": content}

    except Exception as e:
        return {"error": str(e), "predicted_stars": None, "explanation": "", "raw": ""}


print("Testing on first 5 reviews...")
test_results = []
for idx in range(min(5, len(df))):
    row = df.iloc[idx]
    print(f"\n{'='*60}")
    print(f"Review {idx+1}: {row['text'][:100]}...")

    response = classify_review(row['text'])
    print(f"Predicted: {response.get('predicted_stars')}, Actual: {row['stars']}")
    print(f"Explanation: {response.get('explanation', 'N/A')}")
    if 'error' in response:
        print(f"Error: {response['error']}")

    test_results.append(response)

print("\n" + "="*60)
print("If predictions are working, press Enter to continue with full dataset...")
print("If not working, check the raw responses above to debug")
input()

results = []
for idx, row in df.iterrows():
    print(f"Processing {idx+1}/{len(df)}...", end='\r')

    response = classify_review(row['text'])

    results.append({
        "actual_stars": row["stars"],
        "review_text": row["text"],
        "predicted_stars": response.get("predicted_stars"),
        "explanation": response.get("explanation", ""),
        "correct": response.get("predicted_stars") == row["stars"],
        "error": response.get("error", "")
    })

    time.sleep(0.7)

results_df = pd.DataFrame(results)

valid_predictions = results_df[results_df['predicted_stars'].notna()]
accuracy = valid_predictions['correct'].sum() / len(valid_predictions) if len(valid_predictions) > 0 else 0
json_validity = len(valid_predictions) / len(results_df)

print(f"\n\nResults:")
print(f"Total Reviews: {len(results_df)}")
print(f"Valid JSON Rate: {json_validity:.2%}")
print(f"Accuracy: {accuracy:.2%}")
print(f"Correct Predictions: {valid_predictions['correct'].sum()}")

error_count = results_df[results_df['error'] != ''].shape[0]
if error_count > 0:
    print(f"\nErrors encountered: {error_count}")
    print("\nSample errors:")
    print(results_df[results_df['error'] != ''][['error', 'review_text']].head())

results_df.to_csv("classification_results.csv", index=False)
print("\nSaved to: classification_results.csv")

Testing on first 5 reviews...

Review 1: We got here around midnight last Friday... the place was dead. However, they were still serving food...

Raw response: {"predicted_stars": 4, "explanation": "The review is generally positive, mentioning well-made food, friendly service, and a suitable atmosphere, with some neutral comments about the location and late-
Predicted: 4, Actual: 4
Explanation: The review is generally positive, mentioning well-made food, friendly service, and a suitable atmosphere, with some neutral comments about the location and late-night crowd.

Review 2: Brought a friend from Louisiana here.  She says that the crawfish etouffee here is the best she's ha...

Raw response: {"predicted_stars": 5, "explanation": "The reviewer's friend, an out-of-state expert, gave superlative praise to a dish, indicating extremely high satisfaction."}
Predicted: 5, Actual: 5
Explanation: The reviewer's friend, an out-of-state expert, gave superlative praise to a dish, indicating extre


**Prompt 4 (COT)**


In [ ]:
import pandas as pd
import json
import time
from groq import Groq
import re


client = Groq(api_key="apikeyfromgroq")

df = pd.read_csv("/content/test (3).csv")

def get_prompt(review_text):
    prompt = f"""Your task is to Analyze this Yelp review step-by-step. Read each review carefully and understand the opinion being expressed. Based on the content, tone, and overall sentiment of the review, predict how many stars the reviewer likely gave. The rating should be a whole number only, from 1 to 5.

Think through:
1. What positive aspects are mentioned?
2. What negative aspects are mentioned?
3. What is the overall sentiment (very negative, negative, neutral, positive, very positive)?
4. What star rating (1-5) best matches this sentiment?

Review: {review_text}

You MUST respond with ONLY this exact JSON format (no other text before or after):
{{"predicted_stars": <number>, "explanation": "<brief reason>"}}"""
    return prompt

def classify_review(review_text):
    try:
        prompt = get_prompt(review_text)

        response = client.chat.completions.create(
            model="llama-3.3-70b-versatile",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.3,
            max_tokens=150
        )

        content = response.choices[0].message.content.strip()


        print(f"\nRaw response: {content[:200]}")


        try:
            result = json.loads(content)
            if "predicted_stars" in result:
                return result
        except:
            pass

        # Pattern 2: JSON in markdown code blocks
        json_match = re.search(r'```(?:json)?\s*(\{[^`]*\})\s*```', content, re.DOTALL)
        if json_match:
            try:
                result = json.loads(json_match.group(1))
                if "predicted_stars" in result:
                    return result
            except:
                pass


        json_match = re.search(r'\{[^{}]*"predicted_stars"[^{}]*\}', content)
        if json_match:
            try:
                result = json.loads(json_match.group())
                if "predicted_stars" in result:
                    return result
            except:
                pass

        # Pattern 4: Extract numbers if JSON fails
        stars_match = re.search(r'"predicted_stars"\s*:\s*(\d)', content)
        if stars_match:
            return {
                "predicted_stars": int(stars_match.group(1)),
                "explanation": "Extracted from non-standard format"
            }

        return {"error": "No valid JSON found", "predicted_stars": None, "explanation": "", "raw": content}

    except Exception as e:
        return {"error": str(e), "predicted_stars": None, "explanation": "", "raw": ""}


print("Testing on first 5 reviews...")
test_results = []
for idx in range(min(5, len(df))):
    row = df.iloc[idx]
    print(f"\n{'='*60}")
    print(f"Review {idx+1}: {row['text'][:100]}...")

    response = classify_review(row['text'])
    print(f"Predicted: {response.get('predicted_stars')}, Actual: {row['stars']}")
    print(f"Explanation: {response.get('explanation', 'N/A')}")
    if 'error' in response:
        print(f"Error: {response['error']}")

    test_results.append(response)

print("\n" + "="*60)
print("If predictions are working, press Enter to continue with full dataset...")
print("If not working, check the raw responses above to debug")
input()


results = []
for idx, row in df.iterrows():
    print(f"Processing {idx+1}/{len(df)}...", end='\r')

    response = classify_review(row['text'])

    results.append({
        "actual_stars": row["stars"],
        "review_text": row["text"],
        "predicted_stars": response.get("predicted_stars"),
        "explanation": response.get("explanation", ""),
        "correct": response.get("predicted_stars") == row["stars"],
        "error": response.get("error", "")
    })

    time.sleep(0.7)
e
results_df = pd.DataFrame(results)

valid_predictions = results_df[results_df['predicted_stars'].notna()]
accuracy = valid_predictions['correct'].sum() / len(valid_predictions) if len(valid_predictions) > 0 else 0
json_validity = len(valid_predictions) / len(results_df)

print(f"\n\nResults:")
print(f"Total Reviews: {len(results_df)}")
print(f"Valid JSON Rate: {json_validity:.2%}")
print(f"Accuracy: {accuracy:.2%}")
print(f"Correct Predictions: {valid_predictions['correct'].sum()}")


error_count = results_df[results_df['error'] != ''].shape[0]
if error_count > 0:
    print(f"\nErrors encountered: {error_count}")
    print("\nSample errors:")
    print(results_df[results_df['error'] != ''][['error', 'review_text']].head())

results_df.to_csv("classification_results.csv", index=False)
print("\nSaved to: classification_results.csv")

Testing on first 5 reviews...

Review 1: We got here around midnight last Friday... the place was dead. However, they were still serving food...

Raw response: {"predicted_stars": 4, "explanation": "The reviewer mentions several positive aspects, including well-made pub grub, friendly service, quality cocktails, and a great patio, with the only negative bein
Predicted: 4, Actual: 4
Explanation: The reviewer mentions several positive aspects, including well-made pub grub, friendly service, quality cocktails, and a great patio, with the only negative being the place was empty late on a Friday, but this is explained by the location.

Review 2: Brought a friend from Louisiana here.  She says that the crawfish etouffee here is the best she's ha...

Raw response: {"predicted_stars": 5, "explanation": "The reviewer's friend, an apparent connoisseur of Louisiana cuisine, highly praised the crawfish etouffee, indicating an exceptional dining experience."}
Predicted: 5, Actual: 5
Explanation: Th

In [5]:
pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 4.1 MB/s eta 0:00:00


ignore
